# Fine-Tuning VGG16

As another quick baseline model, we'll fine-tune vgg16 since we worked with it in class

## Imports

In [1]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from keras.applications.vgg16 import VGG16


## Loading Data

In [2]:
train_data = np.load('CIFAKE_Train.npz')
test_data = np.load('CIFAKE_Test.npz')


In [3]:
x_train = train_data['images']
y_train = train_data['labels']

# Generate a shuffled index order
num_samples = x_train.shape[0]
shuffled_indices = np.random.permutation(num_samples)

# Use the shuffled indices to shuffle both x_train and y_train
shuffled_x_train = x_train[shuffled_indices]
shuffled_y_train = y_train[shuffled_indices]


In [4]:
x_test = test_data['images']
y_test = test_data['labels']


In [5]:
m_train = x_train.shape[0]
x_train = x_train.reshape(m_train, 32 * 32 * 3)

m_test = x_test.shape[0]
x_test = x_test.reshape(m_test, 32 * 32 * 3)

x_train.shape


(100000, 3072)

We'll normalize the data to be between 0 and 1

In [6]:
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)

x_test = scaler.transform(x_test)

x_train.min(), x_train.max()


(0.0, 1.0)

In [7]:
x_train = x_train.reshape(m_train, 32, 32, 3)
x_test = x_test.reshape(m_test, 32, 32, 3)
x_train.shape


(100000, 32, 32, 3)

## Building the Model

We'll load VGG16 with its ImageNet weights, then replace its classification head with a binary classifier to train on our dataset

In [8]:
vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(32,32,3))


In [9]:
model = Sequential()
model.add(vgg16)
model.add(Flatten())
model.add(Dense(256,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 1, 1, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 256)               131328    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                                 
Total params: 14846273 (56.63 MB)
Trainable params: 14846273 (56.63 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [10]:
model.layers[0].trainable=False


## Training the Model

In [11]:
model.compile(loss='binary_crossentropy',optimizer='Adam',metrics=['accuracy'])


In [12]:
epochs = 8
batch_size = 32
model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2)


Epoch 1/8
2500/2500 [==============================] - 36s 10ms/step - loss: 0.4071 - accuracy: 0.8149 - val_loss: 0.3456 - val_accuracy: 0.8474
Epoch 2/8
2500/2500 [==============================] - 25s 10ms/step - loss: 0.3540 - accuracy: 0.8448 - val_loss: 0.3185 - val_accuracy: 0.8631
Epoch 3/8
2500/2500 [==============================] - 24s 10ms/step - loss: 0.3343 - accuracy: 0.8550 - val_loss: 0.3114 - val_accuracy: 0.8637
Epoch 4/8
2500/2500 [==============================] - 25s 10ms/step - loss: 0.3206 - accuracy: 0.8629 - val_loss: 0.2979 - val_accuracy: 0.8709
Epoch 5/8
2500/2500 [==============================] - 25s 10ms/step - loss: 0.3100 - accuracy: 0.8681 - val_loss: 0.2890 - val_accuracy: 0.8806
Epoch 6/8
2500/2500 [==============================] - 25s 10ms/step - loss: 0.3014 - accuracy: 0.8715 - val_loss: 0.2901 - val_accuracy: 0.8764
Epoch 7/8
2500/2500 [==============================] - 24s 10ms/step - loss: 0.2953 - accuracy: 0.8737 - val_loss: 0.2976 - val_ac

In [13]:
test_loss, test_accuracy = model.evaluate(x_test, y_test)


625/625 [==============================] - 5s 8ms/step - loss: 0.2862 - accuracy: 0.8779


VGG16 did much better than the Logistic Regressor which is expected since it is a CNN on top of a Logistic Regressor. It will be interesting to see how much better other models can do